# CS187

## Lab0-1: Tensors and vectorization

Many of the data-heavy approaches to NLP are enabled by advances in parallel processing that make what were once intractable computations practical. This notebook demonstrates the issue and the `torch` technologies that apply to it, especially the "tensor" data type.

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

In [ ]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs187-2020/lab0-1.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [ ]:
import torch
import random

from timeit import timeit

Numeric vectors can be implemented in Python in many ways. Most directly, Python provides a built-in `list` data type, which we could use to implement a vector. Here, we generate a couple of example vectors as lists each containing 1000 integers between 0 and 99.

In [ ]:
a1 = random.choices(range(100), k=1000)
a2 = random.choices(range(100), k=1000)

### An example: dot product

The dot product of two vectors is the sum of their componentwise product, which can be calculated with a simple for-loop.

In [ ]:
def dotproduct(v1, v2):
    sum = 0
    for i in range(len(v1)):
         sum += v1[i] * v2[i]
    return sum

In [ ]:
dotproduct_result = dotproduct(a1, a2)
dotproduct_result

We can test the efficiency of this approach to implementing vectors by computing a large dot product many times. (We use the `timeit` function that we imported from the `timeit` library to return the time in seconds to perform 100,000 repetitions of the dot product computation.)

In [ ]:
example_time = timeit('dotproduct(a1, a2)', number=100000, globals=globals())
f"It took {example_time} seconds."

As it turns out, performing this vector computation is quite slow -- it probably took several seconds -- because the for loop over the list data structure  computes sequentially. Instead, we can use a data type engineered especially for such vector and array computations to improve performance. Such data types include Python arrays, `numpy` arrays, and [`torch` tensors](https://pytorch.org/docs/stable/tensors.html). The latter are especially designed for the kinds of computations found in machine learning algorithms, so we will use them throughout the course. You can read (a lot) more about tensors in [the official documentation](https://pytorch.org/docs/stable/tensors.html).

We construct a couple of one-dimensional tensors for the examples above.

In [ ]:
t1 = torch.tensor(a1)
t2 = torch.tensor(a2)

### Tensor properties

Tensors have three properties that are especially useful (but potentially confusing when you first start working with them):

+ Componentwise operation: Many operations on tensors work component by component instead of all at once.
+ Broadcast: Operations can broadcast individual elements to each component of a tensor.
+ Reshaping: Tensors can be reshaped to present the same elememtns in different configurations.
+ Special operations: Tensors have mehtods implementing certain operations especially efficiently.

We give examples of each:

#### Componentwise operation

When we add two tensors of the same shape with the `+` operator, the summation percolates down to the individual comonents. For example,

In [ ]:
a3 = [1, 2, 3]
a4 = [4, 5, 6]
t3 = torch.tensor(a3)
t4 = torch.tensor(a4)

t3 + t4

This is quite different from, say, lists, which perform a completely different operation -- concatenation -- when summed with the `+` operator.

In [ ]:
a3 + a4

#### Broadcast

Related, adding a scalar to a tensor "broadcasts" the scalar addition operation to each element.

In [ ]:
print(t3 + 5)
print(5 + t3)

Again, compare with how lists work.

In [ ]:
a3 + 5

#### Reshaping

Finally, tensors can be reshaped so that their elements appear in a different configuration. The `view` method is often used to carry out the reshaping. For instance, we start with the following 3 by 4 tensor.

In [ ]:
t5 = torch.tensor([[11, 12, 13, 14],
                   [21, 22, 23, 24],
                   [31, 32, 33, 34]])

We can view the elements as a 4 by 3 tensor, or a 2 by 2 by 3 tensor, or a 3 by 1 by 4 tensor.

In [ ]:
print(t5.view(4, 3))
print(t5.view(2, 2, 3))
print(t5.view(3, 1, 4))

#### Special operations

Tensors have a large set of methods defined on them that work especially efficiently, for instance, taking the sum of the elements, or the minimum.

In [ ]:
t5.sum()

In [ ]:
t5.min()

We can also take the min/max with respect to a particular dimension. For example, to find the minimum for each row, we can take the min with respect to the second dimension (note that we need to pass 1 since dimension is also 0-indexed). Note that we need to use `.values` since this function would also return the indices where the minimums are (you can check this by using `.indicies` instead of `.values`.)

In [ ]:
t5.min(1).values

<!--
BEGIN QUESTION
name: max_val
-->
Can you find the maximum value of each column?

In [ ]:
# TODO -- Implement a function to return the max value of each column.
def max_col(v):
...

In [ ]:
grader.check("max_val")

### Vectorized dot product

<!--
BEGIN QUESTION
name: dotprod
-->
Using these tensor techniques, and noting the examples above, reimplement a version of `dotproduct` that has no `for` loops. **Hint:** Your code should be *very short*.

In [ ]:
# TODO -- Implement a vectorized dot product, which should be much faster.
def dotproduct_v(v1, v2):
...

In [ ]:
grader.check("dotprod")

This vectorized version should be *much* faster, perhaps a couple of orders of magnitude.

In [ ]:
timeit('dotproduct_v(t1, t2)', number=10000, globals=globals())

### Vectorized computations over multidimensional tensors

Tensors aren't limited to one dimension, and this same vectorization trick applies to multidimensional tensors. In fact, because of vectorization, we can specify computations over multidimensional tensors that look like the kinds of things you've seen in linear algebra. 

#### An example: all-paths shortest path
As a concrete example to give you some practice, consider the algorithm for computing shortest paths in a graph of $n$ nodes. We'll represent the graph as an $n \times n$ matrix $A$ where $A_{ij}$ is the distance from node $i$ to node $j$. (Thus, this is a directed graph, and the distances needn't be symmetric.) Here's an example:

In [ ]:
from math import inf
distances = torch.tensor(
              [[0, 1,   2, 6],
               [1, 0,   2, inf],
               [2, 2,   0, 3],
               [5, inf, 3, 0]])

(We use `inf` for an infinite distance, that is, for nodes that are not connected with an edge.) 

In this graph, the distance from node 0 to node 3 is 6, but by going through node 2, we can shorten the path to 5. In general, consider [the *minplus* operation](https://en.wikipedia.org/wiki/Min-plus_matrix_multiplication) ($\star$) on two square matrices $A$ and $B$:
$$(A \star B)_{ij} = \min_k A_{ik} + B_{kj}$$
If $A$ and $B$ are two graphs over the same nodes (but with different distances), then $A \star B$ is the graph that shows the best way to get from one node to another by traversing an edge from the first graph $B$ and then an edge from the second graph $B$.

Here's an implementation of this operation `minplus` using `for` loops.

In [ ]:
def minplus_loop(A, B):
    R = torch.zeros_like(A)
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert arows == acols and brows == bcols and arows == brows
    for i in range(arows):
        for j in range(arows):
            min = inf
            for k in range(arows):
                if A[i,k] + B[k,j] < min:
                    min = A[i,k] + B[k,j]
                R[i,j] = min
    return R

Using this, we can compute some better ways of getting among the nodes in the`distances` graph. For paths of length at most 2, we can compute

In [ ]:
minplus_loop(distances, distances)

Notice that in this graph, the distance from node 0 to node 3 is now only 5, and there are paths between nodes 1 and 3.

We can compute the minimum distance between any two nodes by repeating this minplus process until no further distance reductions are possible and the graph has reached a stable point (the so-called "fixpoint"). We return the fixpoint graph and the noumber of rounds of minplus that were needed to reach it.

In [ ]:
def minplus_fp(X):
    rounds = 0
    lastY = torch.zeros_like(X)
    Y = X
    while not(torch.equal(Y, lastY)):
        lastY = Y
        Y = minplus_loop(Y, Y)
        rounds += 1
    return Y, rounds

In [ ]:
minplus_fp(distances)

It turns out that after just the two rounds, the fixpoint is reached.

> **Digression:** The complexity of `minplus` as implemented is $O(n^3)$, and the fixpoint computation may need up to $\log n$ calls to `minplus` to converge, so the overall complexity is $O(n^3 \log n)$. More efficient algorithms are known, especially the Floyd-Warshall algorithm for the all-paths versions and Dijkstra's algorithm for the single-source version. But efficiency is not our main aim here.

Let's try a bigger example, a graph with 10 nodes.

In [ ]:
def random_square_tensor(size):
    X = torch.rand(size, size)
    for i in range(size):
        X[i, i] = 0 
    return X

X = random_square_tensor(10)
X

In [ ]:
minplus_fp(X)

Now let's talk about the "loop"-y implementation of the `minplus` function. If we're a little cleverer, we can use list comprehensions to hide the computation of the minimum, but we're still doing the whole computation sequentially.

In [ ]:
def minplus_loop2(A, B):
    R = torch.zeros_like(A)
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert arows == acols and brows == bcols and arows == brows
    for i in range(arows):
        for j in range(arows):
            R[i,j] = min([A[i,k] + B[k,j] for k in range(arows)])
    return R

The `torch`-y way to perform this computation is to rely on vectorized computations. Doing so is a bit tricky however. We need to reshape the matrices a bit. We start by turning the two-dimensional $A$ matrix from a $r \times c$ matrix (rows by columns, which may differ in the general case) into a three-dimensional $r \times 1 \times c$ matrix. Here's the result of that operation on the $4 \times 4$ `distances` matrix, using [the torch `view` method](https://pytorch.org/docs/stable/tensor_view.html).

In [ ]:
distances.view(4, 1, 4)

Now each row in the matrix contains a single element, a vector that corresponds to the column in the original. We'll do a similar operation on $B$ (again, the `distances` matrix in our example, but this time reshaping the matrix to be $1 \times c \times r$.

In [ ]:
distances.view(1, 4, 4)

Now if we add these two matrices componentwise, what do we get? Each of the four first-index elements in $A$ corresponds to only a single first-index element in $B$, so they'll be added componentwise; that single element will be "broadcast" to each of the rows. Thus, the initial element in $A$ (`[[0., 1., 2., 6.]]` in the example) is to be added to the single element in $B$ (`[[0., 1., 2., 6.], [1., 0., 2., inf], [2., 2., 0., 3.], [5., inf, 3., 0.]]` in the example). Proceeding, the same thing happens again, the single element in that $A$ element `[0., 1., 2., 6.]` is broadcast to each of the four elements in $B$ (the first of which is also `[0., 1., 2., 6.]`). Now these are the same size, so they are added elementwise, yielding `[0., 2., 4., 12.]`. Similarly for each of the other three elements in the reshaped $B$ matrix element. When all is said and done, we'd have an $r \times r \times c$ matrix

In [ ]:
summed = distances.view(4, 1, 4) + distances.view(1, 4, 4)
summed

Now we just need to take the minimum of each of the $r \times r$ elements (using the `min` method) to get the final result of the `minplus` operation.

<!--
BEGIN QUESTION
name: minplus_v_example
-->
Calculate the result of the `minplus` by performing appropriate operations on `summed` to yield a tensor that should be identical to the `minplus_loop(distances, distances)` example above.

In [ ]:
#TODO
result = ...
result

In [ ]:
grader.check("minplus_v_example")

Now that you've seen an example of how the matrices can be reshaped and operated on to implement the `minplus` operation, write a function `minplus_v` (a "vectorized" version of `minplus_loop`) that computes the minplus of two matrices without any looping constructs.
<!--
BEGIN QUESTION
name: minplus_v
-->

In [ ]:
#TODO
def minplus_v(A, B):
    ## BEGIN SOLUTION NO PROMPT
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert brows == acols and brows == bcols and arows == brows
    ## END SOLUTION NO PROMPT
    ...

Finally, we'll use your vectorized `minplus_v` to implement a vectorized fixpoint calculation, and test the relative speeds.

In [ ]:
def minplus_vfp(X):
    rounds = 0
    lastY = torch.zeros_like(X)
    Y = X
    while not(torch.equal(Y, lastY)):
        lastY = Y
        Y = minplus_v(Y, Y)
        rounds += 1
    return Y, rounds

In [ ]:
minplus_vfp(distances)

In [ ]:
example = random_square_tensor(20)

timeit('minplus_fp(example)', number=10, globals=globals())

In [ ]:
timeit('minplus_vfp(example)', number=10, globals=globals())

If you've implemented `minplus_v` correctly, the efficiency difference should be striking. This kind of engineered improvement is the difference between a computation taking a day and one taking a minute.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("lab0-1.ipynb")